# Integrated Zebrafish cells
## Xiaonan Wang
## 17July2024

In [ ]:
#%matplotlib nbagg
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd
from os import listdir
from os.path import isfile, join
import re
import anndata
import seaborn as sns

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

from matplotlib.colors import LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list(name='gene_cmap', colors=['lightgrey', 'thistle', 'red', 'darkred']) 

sc.settings.set_figure_params(dpi=80, color_map='viridis', vector_friendly=False,  dpi_save=300)

In [ ]:
adata1 = sc.read('./write/GSE159032.h5ad')
adata2 = sc.read('./write/other.h5ad')

In [ ]:
adata1.obs['Cell_type'].value_counts()

In [ ]:
adata2.obs['Cell_type'].value_counts()

In [ ]:
adata = adata1.concatenate(adata2)

In [ ]:
adata.obs['Cell_type'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sc.pl.umap(adata, color="Cell_type", legend_loc="on data", legend_fontsize="xx-small", ax=ax, show=False)

In [ ]:
# Need to use outer to get exp for all the genes
# GSE138181 - Koth 2020 Development
Koth = sc.read('/cluster/groups/Wangxiaonan/share/rev_MI/GSE138181/write/GSE138181_processed.h5ad', cache=True)
# GSE153170 - Bakker 2021 Development
Bakker = sc.read('/cluster/groups/Wangxiaonan/share/rev_MI/GSE153170/write/GSE153170_processed.h5ad', cache=True)
# GSE172511 - Sun 2022 Circulation
Sun = sc.read('/cluster/groups/Wangxiaonan/share/rev_MI/GSE172511/write/GSE172511_processed.h5ad', cache=True)
# GSE159032 - Hu 2022 Nature genetics
Hu = sc.read('/cluster/groups/Wangxiaonan/share/rev_MI/GSE159032_GSE158919/write/GSE159032_GSE158919_processed_anno.h5ad', cache=True)
# GSE188511 - Kapuria 2022 Development
Kapuria = sc.read('/cluster/groups/Wangxiaonan/share/rev_MI/GSE188511/write/GSE188511_processed.h5ad', cache=True)
# GSE145980 - Ma 2021 EMBO Reports
Ma = sc.read('/cluster/groups/Wangxiaonan/share/rev_MI/GSE145980/write/GSE145980_processed.h5ad', cache=True)

In [ ]:
# adata: .X:row: cell, col: gene -> scaled matrix; .var: high variable genes; .obs: cell annotation
# adata.raw: .X: log norm, .var: all expressed genes
Koth = anndata.AnnData(X=Koth.raw.X, obs=Koth.obs, var=Koth.raw.var, obsm=Koth.obsm)
Bakker = anndata.AnnData(X=Bakker.raw.X, obs=Bakker.obs, var=Bakker.raw.var, obsm=Bakker.obsm)
Sun = anndata.AnnData(X=Sun.raw.X, obs=Sun.obs, var=Sun.raw.var, obsm=Sun.obsm)
Hu = anndata.AnnData(X=Hu.raw.X, obs=Hu.obs, var=Hu.raw.var, obsm=Hu.obsm)
Kapuria = anndata.AnnData(X=Kapuria.raw.X, obs=Kapuria.obs, var=Kapuria.raw.var, obsm=Kapuria.obsm)
Ma = anndata.AnnData(X=Ma.raw.X, obs=Ma.obs, var=Ma.raw.var, obsm=Ma.obsm)

In [ ]:
print('Koth'+str(Koth.shape))
print('Bakker'+str(Bakker.shape))
print('Sun'+str(Sun.shape))
print('Hu'+str(Hu.shape))
print('Kapuria'+str(Kapuria.shape))
print('Ma'+str(Ma.shape))

In [ ]:
adata_raw = [Koth, Bakker, Sun, Hu, Kapuria, Ma]

In [ ]:
for i in range(len(adata_raw)):
    print(adata_raw[i].shape)
    adata_raw[i].var_names_make_unique()

In [ ]:
#combine and save the raw counts
adata_raw = anndata.AnnData.concatenate(*adata_raw, join='outer')

In [ ]:
adata.obs_names = [x[:-2] for x in adata.obs_names]

In [ ]:
adata_raw = adata_raw[adata.obs_names].copy()

In [ ]:
adata.shape

In [ ]:
adata_raw.shape

In [ ]:
adata.raw = adata_raw

In [ ]:
for k in adata.raw.var.columns:
    adata.raw.var[k] = adata.raw.var[k].astype(str)

In [ ]:
adata.write('./adata_paga_int.h5ad')

In [ ]:
adata = sc.read_h5ad('./adata_paga_int.h5ad')

In [ ]:
adata_filtered = adata[adata.obs['Genotype'] == "WT"].copy()
adata_filtered = adata_filtered[~np.in1d(adata_filtered.obs['Treatment'], np.array(['DMSO', 'IWR', 'morphine']))].copy()
adata_filtered = adata_filtered[adata_filtered.obs['Strain'] != "AB"].copy()

In [ ]:
adata_filtered.obs['Condition'].value_counts()

In [ ]:
sc.pl.umap(adata[adata.obs['Condition'] == 'Healthy'], color=['fli1a', 'kdrl', 'myh11a', 'apln'], color_map=cmap, size=10, vmax=2)

In [ ]:
sc.pl.umap(adata[adata.obs['Condition'] == 'Healthy'], legend_fontsize="xx-small", color=['Cell_type'], color_map=cmap, size=10, vmax=2)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sc.pl.umap(adata[adata.obs['Condition'] == 'Healthy'], legend_loc="on data", legend_fontsize="xx-small", color=['lineage.ident'], color_map=cmap,ax=ax, size=10, vmax=2)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sc.pl.umap(adata[adata.obs['Condition'] == 'Healthy'],  legend_fontsize="xx-small", color=['lineage.ident'], color_map=cmap,ax=ax, size=10, vmax=2)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sc.pl.umap(adata[adata.obs['Condition'] == 'Injured'],  legend_fontsize="xx-small", color=['lineage.ident'], color_map=cmap,ax=ax, size=10, vmax=2)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sc.pl.umap(adata[adata.obs['Condition'] == 'Injured'], legend_loc="on data", legend_fontsize="xx-small", color=['lineage.ident'], color_map=cmap,ax=ax, size=10, vmax=2)

# Extract dataset

In [ ]:
tilly_all = adata[adata.obs['StudyID'] == 'GSE138181'].copy()

In [ ]:
tilly_all.obs_names = [x[:-2] for x in tilly_all.obs_names]

In [ ]:
tilly_all.shape

In [ ]:
tilly = sc.read_h5ad("Final_DBpos_norm_afterQC.h5ad")

In [ ]:
tilly.shape

In [ ]:
len(np.intersect1d(tilly.obs_names, tilly_all.obs_names))

In [ ]:
print(tilly.obs_keys())

In [ ]:
p1 = tilly.obs[['louvain_dbpos']]
p2 = tilly_all.obs

In [ ]:
p2.shape

In [ ]:
p2_meta = p2.join(p1, how='left')

In [ ]:
12838-sum(p2_meta['louvain_dbpos'].isnull().values)

In [ ]:
tilly_all.obs = p2_meta

In [ ]:
# Tilly wants to look at the PT, so first calcualte the pseudotime
sc.tl.diffmap(tilly)

In [ ]:
sc.pl.umap(tilly, color=['louvain_dbpos'])

In [ ]:
sc.pl.diffmap(tilly, color=['louvain_dbpos'], color_map=cmap,components=['2,4'])

In [ ]:
dc2 = pd.DataFrame(tilly.obsm['X_diffmap'])[3]
tilly.obs['dpt_root'] = np.in1d(dc2, np.min(dc2))*1

In [ ]:
sc.pl.diffmap(tilly, color=['dpt_root'], color_map=cmap,components=['2,4'])

In [ ]:
sc.pl.umap(tilly, color=['dpt_root'], color_map=cmap)

In [ ]:
tilly.uns['iroot'] = np.where(dc2 == np.min(dc2))[0][0]
sc.tl.dpt(tilly)

In [ ]:
sc.pl.diffmap(tilly, color=['dpt_pseudotime', 'louvain_dbpos'],components=['2,4'], color_map=cmap, save="_oldpc_dbpos.pdf")

In [ ]:
tilly.obs['Condition'] = tilly.obs['Study'].astype(str) + '_' + tilly.obs['louvain_dbpos'].astype(str)

In [ ]:
wt = tilly[tilly.obs['Study'] == 'WT'].copy()

In [ ]:
sc.tl.rank_genes_groups(wt, groupby='Condition', ngenes=wt.raw.shape[1], method='t-test', key_added='DEcon')

In [ ]:
pd.DataFrame(wt.uns['DEcon']['names']).head(10)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(9.5,3), sharex=True, sharey=True)
sc.pl.diffmap(tilly[tilly.obs['Study'] == 'WT'],s=200, vmin=0, vmax=1, color=['louvain_dbpos'],components=['2,4'], color_map=cmap, ax=ax[0], show=False, title="WT")
sc.pl.diffmap(tilly[tilly.obs['Study'] == 'Injured'],s=200, vmin=0, vmax=1, color=['louvain_dbpos'],components=['2,4'], color_map=cmap, ax=ax[1], show=False, title="Injured")
sc.pl.diffmap(tilly[tilly.obs['Study'] == 'Injured_Runx1_KO'],s=200, vmin=0, vmax=1, color=['louvain_dbpos'],components=['2,4'], color_map=cmap, ax=ax[2], show=False, title="Injured_Runx1KO")
plt.tight_layout()
plt.savefig("Diffmap_oldpc_leiden_split.pdf", bbox_inches='tight')

In [ ]:
cmap = LinearSegmentedColormap.from_list(name='gene_cmap', colors=['darkred', 'red', 'orange','blue', 'darkblue']) 

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(9.5,3), sharex=True, sharey=True)
sc.pl.diffmap(tilly[tilly.obs['Study'] == 'WT'],s=200, vmin=0, vmax=1, color=['dpt_pseudotime'],components=['2,4'], color_map=cmap, ax=ax[0], show=False, title="WT")
sc.pl.diffmap(tilly[tilly.obs['Study'] == 'Injured'],s=200, vmin=0, vmax=1, color=['dpt_pseudotime'],components=['2,4'], color_map=cmap, ax=ax[1], show=False, title="Injured")
sc.pl.diffmap(tilly[tilly.obs['Study'] == 'Injured_Runx1_KO'],s=200, vmin=0, vmax=1, color=['dpt_pseudotime'],components=['2,4'], color_map=cmap, ax=ax[2], show=False, title="Injured_Runx1KO")
plt.tight_layout()
plt.savefig("Diffmap_oldpc_dpt_split.pdf", bbox_inches='tight')

In [ ]:
sc.pl.diffmap(tilly[tilly.obs['Study'] == 'WT'],s=200, vmin=0, vmax=1, color=['dpt_pseudotime', 'louvain_dbpos'],components=['2,4'], color_map=cmap, save="_oldpc_dbpos_WT.pdf")

In [ ]:
tilly.obs['Study'].value_counts()

In [ ]:
tilly_all.obs = tilly_all.obs.join(tilly.obs[['dpt_pseudotime']], how='left')

In [ ]:
sc.pl.umap(tilly_all, color='dpt_pseudotime', color_map=cmap)

In [ ]:
sc.pl.umap(tilly_all, color=['Cell_type'], color_map=cmap, s=20)

In [ ]:
tilly.obs = tilly.obs.join(tilly_all.obs[['Cell_type']], how='left')

In [ ]:
sc.pl.umap(tilly_all, color=['dpt_pseudotime', 'louvain_dbpos'], color_map=cmap, s=20, save="_newpc_pt.pdf")

In [ ]:
sc.pl.umap(tilly, color=['dpt_pseudotime'], color_map=cmap, s=100, save="_oldpc_pt.pdf")

In [ ]:
sc.pl.umap(tilly, color=['louvain_dbpos', 'Cell_type'], color_map=cmap, s=100, save="_oldpc_celltype.pdf")

In [ ]:
sc.pl.umap(tilly_all[tilly_all.obs['Index'] == 'GSM4101380'], color='louvain_dbpos', size=30)

In [ ]:
sc.pl.umap(tilly_all[tilly_all.obs['Index'] == 'GSM4101381'], color='louvain_dbpos', size=30)

In [ ]:
sc.pl.umap(tilly_all[tilly_all.obs['Index'] == 'GSM4101382'], color='louvain_dbpos', size=30)